# Live demo - Anthony

# Importaciones necesarias

In [ ]:
import torchvision
import torch
import cv2

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model = model.cuda().eval().half()

# Cargar el modelo y definir cuda para GPU

In [ ]:
model.load_state_dict(torch.load('best_steering_model_xy.pth'))

In [ ]:
device = torch.device('cuda')

# Generar modelo optimizado a partir del anterior (puede tardar)

In [ ]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

In [ ]:
torch.save(model_trt.state_dict(), 'best_steering_model_xy_trt.pth')

# Cargar modelo optimizado

---
Se ajusta la cámara al formato de las imágenes capturadas



In [ ]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_steering_model_xy_trt.pth')) #modelo optimizado

In [ ]:

import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

# Se muestra la cámara

In [ ]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()

image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

# Instancia jetbot

In [ ]:
from jetbot import Robot

robot = Robot()

# Slides para manejo del robot

---

Estas son las diferentes características que se pueden tocar para buscar cuáles son los mejores parámetros para que el robot camine por la carretera. 

El speed gain es para la velocidad a la que va a caminar.
Ambos stering gain slider son para ayudarle a tomar los mejores ángulos. Haciendo pruebas se descrubrió que el más importante es el stering gain, si ambos están en 0 probablemente el robot no tome ninguna vuelta.
El stering bias es en caso de que el robot tenga una tendencia más hacia un lado que hacia el otro, sin embargo, se condideró mejor usar los alphas.

> Nota: Los valores por defecto en que inicia son con los que funcionó bien con nuestro set de datos, puede variar según cada set. Los valores son speed_gain_slider = 0.14 y steering_gain_slider = 0,03. Los demás en 0


In [ ]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.14,description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.03, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

In [ ]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

# Alphas

In [ ]:
robot.left_motor.alpha =2.5
robot.right_motor.alpha = 1.5

# Observador

---

Detectará los cambios de los frames de la cámara.
1. Pre procesa la imagen de la cámara
2. Ejecuta la red neuronal
3. Calcula el valor de la dirección
4. Controla los motores

In [ ]:
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    image = change['new']
    xy = model_trt(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    
execute({'new': camera.value})

Observador de la cámara para que cumpla la función de llamar la función cada vez que recibe un nuevo frame.

In [ ]:
camera.observe(execute, names='value')

# Detención

In [ ]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)

robot.stop()